<a href="https://colab.research.google.com/github/jacobgreen4477/Construction-Equipment-Oil-Condition-Classification-AI-Competition/blob/main/ETRI_v1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> title : 제 4회 ETRI 휴먼이해 인공지능 논문경진대회 <br>
> author : hjy <br>

In [ ]:
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/data/ch2025_data_items/'
ch2025_mACStatus = pd.read_parquet(path+'ch2025_mACStatus.parquet')
ch2025_mActivity = pd.read_parquet(path+'ch2025_mActivity.parquet')
ch2025_mAmbience = pd.read_parquet(path+'ch2025_mAmbience.parquet')
ch2025_mBle = pd.read_parquet(path+'ch2025_mBle.parquet')
ch2025_mGps = pd.read_parquet(path+'ch2025_mGps.parquet')
ch2025_mLight = pd.read_parquet(path+'ch2025_mLight.parquet')
ch2025_mScreenStatus = pd.read_parquet(path+'ch2025_mScreenStatus.parquet')
ch2025_mUsageStats = pd.read_parquet(path+'ch2025_mUsageStats.parquet')
ch2025_mWifi = pd.read_parquet(path+'ch2025_mWifi.parquet')
ch2025_wHr = pd.read_parquet(path+'ch2025_wHr.parquet')
ch2025_wLight = pd.read_parquet(path+'ch2025_wLight.parquet')
ch2025_wPedo = pd.read_parquet(path+'ch2025_wPedo.parquet')